In [1]:
! uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.12.12 environment at: C:\Project\udemy\udemy\LLMOps\.venv
Resolved 67 packages in 286ms
Uninstalled 1 package in 89ms
Installed 1 package in 382ms
 - numpy==2.3.3
 + numpy==2.2.6


In [2]:
import os

from dotenv import load_dotenv

load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


## Data Ingestion


In [3]:
from langchain.document_loaders import TextLoader

In [4]:
from langchain.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(
    "../data",        # 👈 move one level up
    glob="**/*.txt",
    loader_cls=TextLoader
)

documents = loader.load()


In [5]:
documents[0].page_content[:500]  # Print the first 500 characters of the first documen

'AI agents possess several key attributes, including complex goal structures, natural language interfaces, the capacity to act independently of user supervision, and the integration of software tools or planning systems. Their control flow is frequently driven by large language models (LLMs).[2] Agents also include memory systems for remembering previous user-agent interactions and orchestration software for organizing agent components.[3]\nResearchers and commentators have noted that AI agents do'

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

In [8]:
text_chunks=text_splitter.split_documents(documents)

In [9]:
text_chunks

[Document(metadata={'source': '..\\data\\Agentic Ai.txt'}, page_content='AI agents possess several key attributes, including complex goal structures, natural language interfaces, the capacity to act independently of user supervision, and the integration of software tools'),
 Document(metadata={'source': '..\\data\\Agentic Ai.txt'}, page_content='of software tools or planning systems. Their control flow is frequently driven by large language models (LLMs).[2] Agents also include memory systems for remembering previous user-agent interactions'),
 Document(metadata={'source': '..\\data\\Agentic Ai.txt'}, page_content='interactions and orchestration software for organizing agent components.[3]'),
 Document(metadata={'source': '..\\data\\Agentic Ai.txt'}, page_content='Researchers and commentators have noted that AI agents do not have a standard definition.[2][4][5][6] The concept of agentic AI has been compared to the fictional character J.A.R.V.I.S..[7]'),
 Document(metadata={'source': '.

In [10]:
! uv pip install faiss-cpu


Using Python 3.12.12 environment at: C:\Project\udemy\udemy\LLMOps\.venv
Audited 1 package in 4ms


In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [12]:
embeddings=OpenAIEmbeddings()

C:\Users\user\AppData\Local\Temp\ipykernel_23828\2276573386.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings()


In [13]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [14]:
vectorstore

In [15]:
retriever=vectorstore.as_retriever()

In [16]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)


Document 1:
AI agents possess several key attributes, including complex goal structures, natural language interfaces, the capacity to act independently of user supervision, and the integration of software tools
--------------------------------------------------
Document 2:
Researchers and commentators have noted that AI agents do not have a standard definition.[2][4][5][6] The concept of agentic AI has been compared to the fictional character J.A.R.V.I.S..[7]
--------------------------------------------------
Document 3:
A common application of AI agents is the automation of tasksâ€”for example, booking travel plans based on a user's prompted request.[8][9] Prominent examples include Devin AI, AutoGPT, and SIMA.[10]
--------------------------------------------------
Document 4:
Operator.[28] Galileo AI published on Hugging Face a leadership board for agents, which ranks their performance based on their underlying LLMs.[29]
--------------------------------------------------


In [17]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [18]:
prompt=ChatPromptTemplate.from_template(template)

In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [20]:
from langchain.schema.output_parser import StrOutputParser

In [21]:
output_parser=StrOutputParser()

In [22]:
from langchain.chat_models import ChatOpenAI

llm_model=ChatOpenAI(model_name="gpt-4o-mini")

C:\Users\user\AppData\Local\Temp\ipykernel_23828\2671044564.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model=ChatOpenAI(model_name="gpt-4o-mini")


In [23]:
from langchain.schema.runnable import RunnablePassthrough


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [24]:
rag_chain.invoke("tell me about Agentic AI")

'Agentic AI refers to AI agents that have the ability to operate independently, possessing complex goal structures and natural language interfaces. These agents can perform tasks without constant user supervision, making them suitable for automating various applications, such as booking travel plans. Notable examples of agentic AI include Devin AI, AutoGPT, and SIMA. The concept of agentic AI lacks a standard definition and has been likened to fictional characters like J.A.R.V.I.S. Additionally, there are initiatives, such as a leadership board published by Galileo AI on Hugging Face, that rank these agents based on their performance and underlying large language models (LLMs).'

In [25]:
import structlog